In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, gc, re, time
import tqdm as tqdm
import warnings
pd.set_option("max_columns",None)
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
add = pd.read_csv("published_land_price.csv")

In [3]:
add.columns[43:]

Index(['Ｓ５８価格', 'Ｓ５９価格', 'Ｓ６０価格', 'Ｓ６１価格', 'Ｓ６２価格', 'Ｓ６３価格', 'Ｈ１価格', 'Ｈ２価格',
       'Ｈ３価格', 'Ｈ４価格', 'Ｈ５価格', 'Ｈ６価格', 'Ｈ７価格', 'Ｈ８価格', 'Ｈ９価格', 'Ｈ１０価格',
       'Ｈ１１価格', 'Ｈ１２価格', 'Ｈ１３価格', 'Ｈ１４価格', 'Ｈ１５価格', 'Ｈ１６価格', 'Ｈ１７価格', 'Ｈ１８価格',
       'Ｈ１９価格', 'Ｈ２０価格', 'Ｈ２１価格', 'Ｈ２２価格', 'Ｈ２３価格', 'Ｈ２４価格', 'Ｈ２５価格', 'Ｈ２６価格',
       'Ｈ２７価格', 'Ｈ２８価格', 'Ｈ２９価格', 'Ｈ３０価格', 'Ｈ３１価格', '属性移動Ｓ５９', '属性移動Ｓ６０',
       '属性移動Ｓ６１', '属性移動Ｓ６２', '属性移動Ｓ６３', '属性移動Ｈ１', '属性移動Ｈ２', '属性移動Ｈ３', '属性移動Ｈ４',
       '属性移動Ｈ５', '属性移動Ｈ６', '属性移動Ｈ７', '属性移動Ｈ８', '属性移動Ｈ９', '属性移動Ｈ１０', '属性移動Ｈ１１',
       '属性移動Ｈ１２', '属性移動Ｈ１３', '属性移動Ｈ１４', '属性移動Ｈ１５', '属性移動Ｈ１６', '属性移動Ｈ１７',
       '属性移動Ｈ１８', '属性移動Ｈ１９', '属性移動Ｈ２０', '属性移動Ｈ２１', '属性移動Ｈ２２', '属性移動Ｈ２３',
       '属性移動Ｈ２４', '属性移動Ｈ２５', '属性移動Ｈ２６', '属性移動Ｈ２７', '属性移動Ｈ２８', '属性移動Ｈ２９',
       '属性移動Ｈ３０', '属性移動Ｈ３１'],
      dtype='object')

In [4]:
rename_dict1 = {'経度':"Longitude", "緯度":"Latitude", "所在地コード":"Adress_code", "用途":"Use", "連番":"Serial_number"}

In [5]:
rename_dict1 = {'経度':"Longitude", "緯度":"Latitude", "所在地コード":"Adress_code", "用途":"Use", "連番":"Serial_number",
                "年次":"Year", "前年所在地コード":"Before_adress_code", "前年用途":"Before_use", "前年連番":"Before_serial_number"}

rename_dict2 = {'市区町村名':"Municipality", "住居表示":"Adress", "行政":"Administration", "地積":"Area", "利用の現況":"Current_usage",
                "利用状況表示":"Current_usage_text", "建物構造":"Structure", "施設":"Infrastructure", "形状区分":"Shape",
               "間口（比率）":"Frontage"}

rename_dict3 = {'奥行（比率）':"Depth", "階層（地上）":"Hierarchy_surface", "階層（地下）":"Hierarchy_under",
                "前面道路区分":"Front_road", "前面道路の方位区分":"Front_road_direction", "前面道路の幅員":"Front_road_width",
                "前面道路の駅前区分":"Front_road_station", "前面道路の舗装状況":"Front_road_state", "側道区分":"Side_road_state"}

rename_dict4 = {'側道方位区分':"Side_road_direction", "交通施設との近接区分":"Proximity_traffic",
                "周辺の土地の利用の現況":"Surround_usage", "駅名":"Station", "駅距離":"Station_distance", 
                "用途区分":"Usage_Classification", "防火区分":"Fire_classification", "都市計画区分":"CityPlanning_classification",
                "森林区分":"Forest_classification", "公園区分":"Park_classification", "建蔽率":"Coverage"}

rename_dict5 = {'容積率':"Volume", "共通地点区分":"Common_classification", "選定年次ビット":"Year_bid"}

In [6]:
cols = ['Ｓ５８価格', 'Ｓ５９価格', 'Ｓ６０価格', 'Ｓ６１価格', 'Ｓ６２価格', 'Ｓ６３価格']
cols2 = ['Ｈ１価格', 'Ｈ２価格','Ｈ３価格', 'Ｈ４価格', 'Ｈ５価格', 'Ｈ６価格', 'Ｈ７価格', 'Ｈ８価格', 'Ｈ９価格', 'Ｈ１０価格',
         'Ｈ１１価格', 'Ｈ１２価格', 'Ｈ１３価格', 'Ｈ１４価格', 'Ｈ１５価格', 'Ｈ１６価格', 'Ｈ１７価格', 'Ｈ１８価格',
         'Ｈ１９価格', 'Ｈ２０価格', 'Ｈ２１価格', 'Ｈ２２価格', 'Ｈ２３価格', 'Ｈ２４価格', 'Ｈ２５価格', 'Ｈ２６価格',
         'Ｈ２７価格', 'Ｈ２８価格', 'Ｈ２９価格', 'Ｈ３０価格', 'Ｈ３１価格']
cols3 = ['属性移動Ｓ５９', '属性移動Ｓ６０', '属性移動Ｓ６１', '属性移動Ｓ６２', '属性移動Ｓ６３']
cols4 = ['属性移動Ｈ１', '属性移動Ｈ２', '属性移動Ｈ３', '属性移動Ｈ４', '属性移動Ｈ５', '属性移動Ｈ６',
         '属性移動Ｈ７', '属性移動Ｈ８', '属性移動Ｈ９', '属性移動Ｈ１０', '属性移動Ｈ１１', '属性移動Ｈ１２',
         '属性移動Ｈ１３', '属性移動Ｈ１４', '属性移動Ｈ１５', '属性移動Ｈ１６', '属性移動Ｈ１７','属性移動Ｈ１８', '属性移動Ｈ１９',
         '属性移動Ｈ２０', '属性移動Ｈ２１', '属性移動Ｈ２２', '属性移動Ｈ２３', '属性移動Ｈ２４', '属性移動Ｈ２５',
         '属性移動Ｈ２６', '属性移動Ｈ２７', '属性移動Ｈ２８', '属性移動Ｈ２９', '属性移動Ｈ３０', '属性移動Ｈ３１']
rename_price = {}

for i, c in enumerate(cols):
    rename_price[c] = "S"+str(i+58)+"_Price"
for i, c in enumerate(cols2):
    rename_price[c] = "H"+str(i+1)+"_Price"
for i, c in enumerate(cols3):
    rename_price[c] = "S"+str(i+59)+"_Attribute"
for i, c in enumerate(cols4):
    rename_price[c] = "H"+str(i+1)+"_Attribute"

In [7]:
add = add.rename(columns=rename_dict1)
add = add.rename(columns=rename_dict2)
add = add.rename(columns=rename_dict3)
add = add.rename(columns=rename_dict4)
add = add.rename(columns=rename_dict5)
add = add.rename(columns=rename_price)

In [8]:
add["Adress"] = add["Adress"].apply(lambda x: re.sub("１","1",x))
add["Adress"] = add["Adress"].apply(lambda x: re.sub("２","2",x))
add["Adress"] = add["Adress"].apply(lambda x: re.sub("３","3",x))
add["Adress"] = add["Adress"].apply(lambda x: re.sub("４","4",x))
add["Adress"] = add["Adress"].apply(lambda x: re.sub("５","5",x))
add["Adress"] = add["Adress"].apply(lambda x: re.sub("６","6",x))
add["Adress"] = add["Adress"].apply(lambda x: re.sub("７","7",x))
add["Adress"] = add["Adress"].apply(lambda x: re.sub("８","8",x))
add["Adress"] = add["Adress"].apply(lambda x: re.sub("９","9",x))
add["Adress"] = add["Adress"].apply(lambda x: re.sub("０","0",x))
add["Adress"] = add["Adress"].apply(lambda x: re.sub("－","-",x))

In [9]:
add["Adress"] = add["Adress"].apply(lambda x: x.split("\u3000")[1])
add["Adress"] = add["Adress"].apply(lambda x: re.split("[0-9]+-", x, 1)[0])
add["Adress"] = add["Adress"].apply(lambda x: re.split("[0-9]+番", x, 1)[0])
add["Adress"] = add["Adress"].apply(lambda x: re.split("[0-9]", x, 1)[0])

In [10]:
add.to_csv("land_price.csv",index=False)